## Main nb

As a prof of content we will just bet on the date

In [3]:
from functions import *
# Example usage for testing
if __name__ == "__main__":
    bet_date = '2024/09/25'  # Date for the bet
    bet_amount = 10  # Amount placed for the bet
    csv_file = 'betting_odds.csv'  # CSV file with betting odds and statistics
    ledger_file = 'global_ledger.csv'  # CSV file with global ledger (historical bets)

    print("Welcome to the betting odds program! Choose a date to get the odds for the bet.")
    odds = get_odds(csv_file)
    
    new_odd=input("Press 1 to get odd for another date")
    while new_odd=='1':
        bet_date = input("Enter the date for the bet in the format YYYY/MM/DD: ")
        odds = get_odds(csv_file, bet_date)
        new_odd=input("Press 1 to get odd for another date")
    # Update the betting odds file after a bet is placed
    update_betting_file(bet_date, bet_amount, csv_file, ledger_file)


Welcome to the betting odds program! Choose a date to get the odds for the bet.
Odds: 2.8665
Odds: 2.8753


ValueError: time data "269" at position 0 doesn't match format specified

In [ ]:

mu_0 = 100 # day of year  
std_0 = 5   

"""
We will use a normal distribution to model the probability of the day of the year of the break up. 
This distribution will be 'bayesianly' updated with the data of the current bets , and the uncertainty will decrease as the date of the break up approaches. 
"""
base_pdf = stats.norm(mu_0, std_0)
#t
bets = []  

plot_pdf(base_pdf, "Initial PDF")

